In [20]:
import pandas as pd
import random, secrets
import numpy as np
import os, sys
from faker import Faker
import json
import datetime
from localutils import simsystemhelper
import networkx as nx
from typing import List, Dict, Any, Tuple, Set

In [17]:
ospw = simsystemhelper.os_p_weights
sppw = simsystemhelper.system_posture_p_weights
win_servers = simsystemhelper.windows_servers
lin_servers = simsystemhelper.linux_servers


server = simsystemhelper.SimSystem(ospw, sppw, win_servers, lin_servers)

In [18]:
server.generate_random_system()

{'hostname': 'L-email-97.hal.com',
 'group': 'REV',
 'os': 'Linux',
 'os_ver': 'Ubuntu 22.04',
 'posture': {'PublicExposure': True,
  'AccessPublic': True,
  'CriticalDataProcess': True,
  'OpenSourceComponents': True,
  'PatchMgmt': True,
  'IncidentMgmt': True,
  'ChangeMgmt': False,
  'StrongAuth': False,
  'AccessControl': False,
  'AppFirewall': True,
  'VulnMgmt': True,
  'DataEnc': True,
  'DataClass': True,
  'DataBackup': True,
  'DataLossPrevention': True,
  'Monitoring': False,
  'Logging': True,
  'NetworkSeg': True,
  'NetworkFirewall': True,
  'SIEM': True,
  'Location': True}}

In [ ]:
class AttackChainAnalyzer:
    """
    Analyzes attack chains by constructing a directed graph of vulnerabilities,
    incorporating CWE relationships to enhance attack path analysis.
    """
    
    def __init__(self, df: pd.DataFrame) -> None:
        """
        Initializes the attack chain analyzer.
        
        :param df: Pandas DataFrame containing CVE data with 'cve_id', 'impact', 'severity', and 'cwe'.
        """
        self.df = df
        self.attack_graph = nx.DiGraph()
        self.build_graph()

    def build_graph(self) -> None:
        """
        Builds a directed graph of potential attack chains based on impact levels and CWE relationships.
        """
        impact_order = {
            "RCE": 4,
            "Privilege Escalation": 3,
            "Auth Bypass": 2,
            "Lateral Movement": 1
        }
        
        # Add vulnerabilities as nodes with CWE
        for _, vuln in self.df.iterrows():
            self.attack_graph.add_node(
                vuln["cve_id"], impact=vuln["impact"], severity=vuln["severity"], cwe=vuln.get("cwe", "Unknown")
            )

        # Connect vulnerabilities based on impact levels and CWE relationships
        for _, high_vuln in self.df.iterrows():
            for _, low_vuln in self.df.iterrows():
                if high_vuln["cve_id"] != low_vuln["cve_id"]:
                    if impact_order.get(high_vuln["impact"], 0) > impact_order.get(low_vuln["impact"], 0):
                        self.attack_graph.add_edge(high_vuln["cve_id"], low_vuln["cve_id"])
                    elif high_vuln.get("cwe") == low_vuln.get("cwe"):  # CWE similarity linking
                        self.attack_graph.add_edge(high_vuln["cve_id"], low_vuln["cve_id"])

    def find_unique_chains(self) -> List[Tuple[str, ...]]:
        """
        Identifies unique attack paths from high-impact vulnerabilities to lower ones.
        
        :return: List of unique attack paths represented as tuples of CVE IDs.
        """
        unique_chains: Set[Tuple[str, ...]] = set()
        for start_node in self.attack_graph.nodes:
            for end_node in self.attack_graph.nodes:
                if start_node != end_node:
                    paths = list(nx.all_simple_paths(self.attack_graph, source=start_node, target=end_node))
                    unique_chains.update(tuple(path) for path in paths)
        return sorted(unique_chains, key=len, reverse=True)
    
    def save_chains_to_csv(self, filename: str) -> str:
        """
        Saves the unique attack chains to a CSV file.
        
        :param filename: Name of the output CSV file.
        :return: File path of the saved CSV file.
        """
        chains = self.find_unique_chains()
        df = pd.DataFrame(chains, columns=[f"Step {i+1}" for i in range(len(max(chains, key=len)))])
        filepath = os.path.join(OUTPUT_FOLDER, filename)
        df.to_csv(filepath, index=False)
        return filepath
    
    def save_chains_to_json(self, filename: str) -> str:
        """
        Saves the unique attack chains to a JSON file.
        
        :param filename: Name of the output JSON file.
        :return: File path of the saved JSON file.
        """
        chains = self.find_unique_chains()
        filepath = os.path.join(OUTPUT_FOLDER, filename)
        with open(filepath, 'w') as json_file:
            json.dump(chains, json_file, indent=4)
        return filepath